## In this notebook, we will transform multiple tables into embedded document in json format for importing into mongodb

In [1]:
import pandas as pd
from datetime import datetime
from langdetect import detect
import math
from tqdm.notebook import tqdm
import json

In [2]:
# import libraries and run a few checks
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import pandas as pd
import scipy.stats as stats
import os


import warnings 
warnings.filterwarnings('ignore')

# util to reduce memory usage
## From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Memory usage decreased to {end_mem:.2f} Mb ({1-end_mem/start_mem:.2%} reduction)')
    return df



In [3]:
# importing cleaned and generated data
calendar_df = pd.read_csv('calendar_cleaned.csv',  encoding="utf-8")
listings_df = pd.read_csv('listings_cleaned.csv',  encoding="utf-8")
reviews_df = pd.read_csv('reviews_cleaned.csv',  encoding="utf-8")
customers_df = pd.read_csv('customers.csv',  encoding="utf-8")
transactions_df = pd.read_csv('transactions.csv',  encoding="utf-8")

In [4]:
#create host table based on our schema
host_df = listings_df[['host_id', 'host_name', 'host_url', 'host_since','host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood','host_has_profile_pic','host_identity_verified','host_verifications']].copy()

In [8]:
test_df = pd.DataFrame(listings_df.groupby(['host_id'])['id'].nunique())

In [15]:
test_df.sort_values(by=['id'], ascending = False)

,id
host_id,
201677068,195
39669202,102
52161947,81
26736483,71
187786849,65
...,...
79482033,1
79459332,1
79244729,1


In [17]:
test_df.loc[test_df['id'] == 1].count()/test_df.shape[0]

id    0.700223
dtype: float64

In [19]:
test2_df = pd.DataFrame(listings_df.groupby(['id'])['host_id'].nunique())

In [22]:
test2_df.loc[test2_df['host_id'] != 1].count()

host_id    0
dtype: int64

In [5]:
listings_df = listings_df[['id','listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
                'neighborhood_overview', 'picture_url', 'host_id','neighbourhood','neighbourhood_cleansed',
                'latitude', 'longitude', 'property_type','room_type', 'accommodates', 'bathrooms_text', 
                'bedrooms', 'beds', 'calendar_last_scraped', 'review_scores_rating', 'review_scores_accuracy',
                'review_scores_cleanliness', 'review_scores_checkin',
                'review_scores_communication', 'review_scores_location',
                'review_scores_value', 'instant_bookable']]

### Queries

Show the listings available for booking from X date for Y days. User can select optional filters (can just demonstrate three as example?):​

	Accommodates X people​

	Price within X range (total price for that period, referencing calendar table)​

	Review score rating greater than X​

	Property type​

	Etc.​

Demonstrate join with host table:​

	Show the listings where the host is a superhost​

	Demonstrate join with amenities table:​

	e.g. Show the listings with WiFi​

Demonstrate join with reviews table:​

	Show the reviews for listing X​
  
5. Average Check in/ average price past 12 months for certain listings​

6. Calculate total listings of certain host​

7. total revenue of a certain host for certain period (transaction join listing join host)​

8. find listings with at least one review (review score ratings is not null)



#### transforming calendar by dates, group by listing id, for embedded documents, merge to listings using listing id
listing_id : "dates":[{"date" : "2022-06-20",
                      "available": "t",
                      "price": 890,
                      "adjusted_price": 890,
                      "minimum_nights": 1,
                      "maximum_nights": 365,
                      },
                      ]

In [8]:
import re
def concatstring(list1, cols):
    numerics = ['minimum_nights', 
                'maximum_nights', 
                'host_id', 
                'review_id', 
                'customer_id', 
                'transaction_id', 
                'length_of_stay', 
                'cost_per_day', 
                'total_cost', 
                'number_of_guests', 
                'refund_percentage']
    
    string = "{"
    for i,item in enumerate(list1):
        
        if str(item) == 'nan':
            string = string + "\"" + cols[i] + "\" : " + "null" + ","
            
        else:
            
            if type(item) != 'str':
                item = str(item)
                
            if cols[i] in numerics:
                string = string + "\"" + cols[i] + "\" : " + item + ","
            elif item.startswith('$') or 'price' in cols[i]:
                string = string + "\"" + cols[i] + "\" : " + item.strip('$').replace(",","") + ","
            elif item.endswith('%') or 'rate' in cols[i]:
                string = string + "\"" + cols[i] + "\" : " + item.strip('%') + ","
            elif re.match(r'\d{4}-\d{2}-\d{2}', item):
            #    string = string + "\"" + cols[i] + "\" : " + "ISODate(\"" + item + "T00:00:00.000Z" + "\")," 
                string = string + "\"" + cols[i] + "\" : " + "\"" + item + "T00:00:00.000Z" + "\"," 
            elif item.endswith(']') and item.startswith('['):
                string = string + "\"" + cols[i] + "\" : " + item.replace("\'","\"") + ","
            else:
                string = string + "\"" + cols[i] + "\" : " + "\"" + item + "\","

    string = string + "}"
    
    return string
    

In [28]:
calendar_df = pd.read_csv('calendar_cleaned.csv',  encoding="utf-8")

In [29]:
cols = calendar_df.columns.tolist()
cols.remove('listing_id')
calendar_df['dates'] = calendar_df.apply(lambda x: concatstring([x.date, 
                                                                 x.available, 
                                                                 x.price, 
                                                                 x.adjusted_price, 
                                                                 x.minimum_nights, 
                                                                 x.maximum_nights],cols), axis = 1)

In [30]:
calendar_df = calendar_df.groupby(['listing_id'])['dates'].agg(lambda x: ",".join(str(v)[:-2] + "}" for v in x))

In [31]:
calendar_df = pd.DataFrame({'listing_id':calendar_df.index, 'dates':calendar_df.values})

In [32]:
calendar_df['dates'] = calendar_df['dates'].apply(lambda x: "[" + x + "]")

In [33]:
calendar_df.iloc[1]['dates']

'[{"date" : "2022-06-21","available" : "f","price" : 1342.0,"adjusted_price" : 1342.0,"minimum_nights" : 1.0,"maximum_nights" : 730.0},{"date" : "2022-06-22","available" : "f","price" : 1342.0,"adjusted_price" : 1342.0,"minimum_nights" : 1.0,"maximum_nights" : 730.0},{"date" : "2022-06-23","available" : "f","price" : 1342.0,"adjusted_price" : 1342.0,"minimum_nights" : 1.0,"maximum_nights" : 730.0},{"date" : "2022-06-24","available" : "f","price" : 1342.0,"adjusted_price" : 1342.0,"minimum_nights" : 1.0,"maximum_nights" : 730.0},{"date" : "2022-06-25","available" : "f","price" : 1342.0,"adjusted_price" : 1342.0,"minimum_nights" : 1.0,"maximum_nights" : 730.0},{"date" : "2022-06-26","available" : "f","price" : 1342.0,"adjusted_price" : 1342.0,"minimum_nights" : 1.0,"maximum_nights" : 730.0},{"date" : "2022-06-27","available" : "f","price" : 1342.0,"adjusted_price" : 1342.0,"minimum_nights" : 1.0,"maximum_nights" : 730.0},{"date" : "2022-06-28","available" : "t","price" : 1342.0,"adjusted

In [34]:
calendar_df.to_csv('calendar_df_transformed.csv', index = False, columns = calendar_df.columns.tolist())

####  transforming host by host ids, merge to listings using host_id

In [35]:
cols = host_df.columns.tolist()
host_df['hosts'] = host_df.apply(lambda x: concatstring([x.host_id,
                                                        x.host_name,
                                                        x.host_url,
                                                        x.host_since,
                                                        x.host_location,
                                                        x.host_about,
                                                        x.host_response_time,
                                                        x.host_response_rate,
                                                        x.host_acceptance_rate,
                                                        x.host_is_superhost,
                                                        x.host_thumbnail_url,
                                                        x.host_picture_url,
                                                        x.host_neighbourhood,
                                                        x.host_has_profile_pic,
                                                        x.host_identity_verified,
                                                        x.host_verifications],cols), axis = 1)

In [36]:
host_df = host_df.drop_duplicates()

In [37]:
host_df = pd.DataFrame({'host_id':host_df['host_id'], 'hosts':host_df['hosts']})
host_df['hosts'] = host_df['hosts'].apply(lambda x: x[:-2] + "}")

In [38]:
host_df.iloc[2]['hosts']

'{"host_id" : 303573998,"host_name" : "Noodee","host_url" : "https://www.airbnb.com/users/show/303573998","host_since" : "2019-10-21","host_location" : "TH","host_about" : null,"host_response_time" : "a few days or more","host_response_rate" : 0.0,"host_acceptance_rate" : null,"host_is_superhost" : "f","host_thumbnail_url" : "https://a0.muscache.com/im/pictures/user/3ed3b465-3c37-4737-a4d0-b865ec8f31b4.jpg?aki_policy=profile_small","host_picture_url" : "https://a0.muscache.com/im/pictures/user/3ed3b465-3c37-4737-a4d0-b865ec8f31b4.jpg?aki_policy=profile_x_medium","host_neighbourhood" : null,"host_has_profile_pic" : "t","host_identity_verified" : "t","host_verifications" : ["email", "phone"]}'

In [39]:
host_df.to_csv('host_df_transformed.csv', index = False, columns = host_df.columns.tolist())

#### merge review and transaction and customer, then transform by reviewer ids, merge to listings using listing_id

In [18]:
transactions_df = pd.read_csv('transactions.csv',  encoding="utf-8")
reviews_df = pd.read_csv('reviews_cleaned.csv',  encoding="utf-8")
customers_df = pd.read_csv('customers.csv',  encoding="utf-8")

In [19]:
reviews_df = reviews_df.merge(transactions_df, left_on = ['id', 'listing_id'], right_on = ['review_id','listing_id'], how = 'left')

In [20]:
reviews_df = reviews_df.merge(customers_df,on = ['customer_id', 'customer_name'], how = 'right')

In [21]:
reviews_df.columns.tolist()

['listing_id',
 'id',
 'date',
 'reviewer_id',
 'reviewer_name',
 'comments',
 'review_id',
 'customer_id',
 'review_date',
 'customer_name',
 'transaction_id',
 'check_out_date',
 'length_of_stay',
 'check_in_date',
 'transaction_date',
 'cost_per_day',
 'total_cost',
 'number_of_guests',
 'cancelled_transaction',
 'refund_percentage',
 'phone',
 'email']

In [24]:
cols = reviews_df.columns.tolist()
cols.remove('listing_id')
cols.remove('id')
reviews_df['transactions'] = reviews_df.apply(lambda x: concatstring([x.date,
                                                                   x.reviewer_id,
                                                                   x.reviewer_name,
                                                                   x.comments,
                                                                   x.review_id,
                                                                   x.customer_id,
                                                                   x.review_date,
                                                                   x.customer_name,
                                                                   x.transaction_id,
                                                                   x.check_out_date,
                                                                   x.length_of_stay,
                                                                   x.check_in_date,
                                                                   x.transaction_date,
                                                                   x.cost_per_day,
                                                                   x.total_cost,
                                                                   x.number_of_guests,
                                                                   x.cancelled_transaction,
                                                                   x.refund_percentage,
                                                                   x.phone,
                                                                   x.email],cols), axis = 1)

In [25]:
reviews_df = reviews_df.groupby(['listing_id'])['transactions'].agg(lambda x: ",".join(str(v)[:-2] + "}" for v in x))
reviews_df = pd.DataFrame({'listing_id':reviews_df.index, 'transactions':reviews_df.values})
reviews_df['transactions'] = reviews_df['transactions'].apply(lambda x: "[" + x + "]")

In [26]:
reviews_df.iloc[3]['transactions']

'[{"date" : "2011-03-16","reviewer_id" : "371435","reviewer_name" : "Alan","comments" : "decent rooms friendly staff would book again","review_id" : null,"customer_id" : null,"review_date" : null,"customer_name" : null,"transaction_id" : null,"check_out_date" : null,"length_of_stay" : null,"check_in_date" : null,"transaction_date" : null,"cost_per_day" : null,"total_cost" : null,"number_of_guests" : null,"cancelled_transaction" : null,"refund_percentage" : null,"phone" : null,"email" : null},{"date" : "2011-06-13","reviewer_id" : "142645","reviewer_name" : "Christine","comments" : "I had a wonderful stay at sathorn terrace. Bangkok is such a big city and staying in a place that is centrally located is very key. Tor was such a wonderful host and such a big help in recommendations around the city. he went above and beyond to make our stay so comfortable and pleasant. thank you, tor!!\xa0\r<br/>","review_id" : null,"customer_id" : null,"review_date" : null,"customer_name" : null,"transact

In [27]:
reviews_df.to_csv('reviews_df_transformed.csv', index = False, columns = reviews_df.columns.tolist())

#### merge [calendar_df, host_df, reviews_df] to listings

In [28]:
calendar_df = pd.read_csv('calendar_df_transformed.csv',  encoding="utf-8")
reviews_df = pd.read_csv('reviews_df_transformed.csv',  encoding="utf-8")
host_df = pd.read_csv('host_df_transformed.csv',  encoding="utf-8")

In [29]:
listings_df = pd.read_csv('listings_cleaned.csv',  encoding="utf-8")
listings_df = listings_df[['id','listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
                'neighborhood_overview', 'picture_url', 'host_id','neighbourhood','neighbourhood_cleansed',
                'latitude', 'longitude', 'property_type','room_type', 'accommodates', 'bathrooms_text', 
                'bedrooms', 'beds', 'calendar_last_scraped', 'review_scores_rating', 'review_scores_accuracy',
                'review_scores_cleanliness', 'review_scores_checkin',
                'review_scores_communication', 'review_scores_location',
                'review_scores_value', 'instant_bookable']]
listings_df = listings_df.rename(columns = {"id":"listing_id"})

In [30]:
listings_df = listings_df.merge(calendar_df[['listing_id', 'dates']], on = 'listing_id', how = 'left' )

In [31]:
listings_df = listings_df.merge(host_df[['host_id', 'hosts']], on = 'host_id', how = 'left' )

In [32]:
listings_df = listings_df.merge(reviews_df[['listing_id', 'transactions']], on = 'listing_id', how = 'left' )

In [33]:
listings_df.shape[0]

17074

In [34]:
listings_df.to_csv('transformed.csv', index = False, columns = listings_df.columns.tolist())

In [36]:
listings_df

,listing_id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,neighbourhood,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,dates,hosts,transactions
0,645048906856594097,https://www.airbnb.com/rooms/645048906856594097,20220620202126,2022-06-20,Furnished 1 bedroom condo with beautiful view,🏢Condo For rent: Lumpini Ville Nakhon In-River...,Its in Lumpini building,https://a0.muscache.com/pictures/miso/Hosting-...,4663219,"Tambon Talat Kwan, Chang Wat Nonthaburi, Thailand",...,NaN,NaN,NaN,NaN,NaN,NaN,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 4663219,""host_name"" : ""Fawad"",""ho...",NaN
1,15636062,https://www.airbnb.com/rooms/15636062,20220620202126,2022-06-20,Akesin place ngamwongwan,ที่ของฉันใกล้กับร้านอาหารและของกิน และ ขนส่งสา...,NaN,https://a0.muscache.com/pictures/f68098b9-3ff2...,100840975,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 100840975,""host_name"" : ""Supika"",...",NaN
2,39544047,https://www.airbnb.com/rooms/39544047,20220620202126,2022-06-20,IMPACT Thunder Dome Muangthong ND condo ( C1 ),NaN,IMPACT ศูนย์แสดงสินค้า 550 m<br />IMPACT Chal...,https://a0.muscache.com/pictures/cc410e7d-922f...,303573998,"Tambon Ban Mai, จ.นนทบุรี, Thailand",...,5.0,5.0,5.0,5.0,4.67,4.67,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 303573998,""host_name"" : ""Noodee"",...","[{""date"" : ""2019-11-15"",""reviewer_id"" : ""27394..."
3,34002624,https://www.airbnb.com/rooms/34002624,20220620202126,2022-06-20,Popular Condo Near IMPACT,คอนโดเมืองทองอยู่ในบริเวณเดียวกับศูนย์แสดงสินค...,คอนโดเมืองทองอยู่ในบริเวณเดียวกับศูนย์แสดงสินค...,https://a0.muscache.com/pictures/e4811699-0153...,256747704,"ตำบล บางพูด, Chang Wat Nonthaburi, Thailand",...,5.0,5.0,5.0,4.0,4.50,4.00,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 256747704,""host_name"" : ""Krit"",""h...","[{""date"" : ""2019-06-01"",""reviewer_id"" : ""25693..."
4,22678899,https://www.airbnb.com/rooms/22678899,20220620202126,2022-06-20,บ้านเบนซ์,เป็นหมู่บ้านขนาดใหญ่มีหลายเฟส บ้านเป็นทาวน์โฮม...,NaN,https://a0.muscache.com/pictures/02e4ae53-dbe6...,167136634,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 167136634,""host_name"" : ""เบนซ์"",""...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17069,22947580,https://www.airbnb.com/rooms/22947580,20220620202126,2022-06-20,Apartment T9 Muang Thong Thani by KhunNutt2B7,ห้องพักส่วนตัวสวยหรูอยู่สบาย สะอาด ราคาประหยัด...,ที่พักอยู่ใกล้ห้างคอสโม แหล่งช้อปปิ้ง Sport Fa...,https://a0.muscache.com/pictures/2cbf3a10-76e8...,169985358,"Tambon Ban Mai, Chang Wat Nonthaburi, Thailand",...,NaN,NaN,NaN,NaN,NaN,NaN,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 169985358,""host_name"" : ""Nath"",""h...",NaN
17070,5955949,https://www.airbnb.com/rooms/5955949,20220620202126,2022-06-20,Cozy Manor house with Lake view 520 square meters,Nice place for relax with wide lake at opposit...,There are many Thai restaurant near entrance o...,https://a0.muscache.com/pictures/93a7608f-f35c...,20002101,"Tambon Bang Nam Chut, Chang Wat Samut Sakhon, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,f,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 20002101,""host_name"" : ""Nutchuda""...",NaN
17071,32650300,https://www.airbnb.com/rooms/32650300,20220620202126,2022-06-20,Studio Superior (Double Bed) Room 1,- Room size 25 sq.m. or 322 sq.ft.<br />- Quee...,"Bangna Tower<br />40,000 square meters project...",https://a0.muscache.com/pictures/9a88370c-111c...,245251233,"บางแก้ว, Samut Prakan, Thailand",...,NaN,NaN,NaN,NaN,NaN,NaN,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" :

In [37]:
listings_df2 = listings_df.fillna("")

### Change to json then compass import mongodb

In [38]:
listings_df = pd.read_csv('transformed.csv',  encoding="utf-8")

In [39]:
listings_df2 = listings_df.fillna("")
listings_df2

,listing_id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,neighbourhood,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,dates,hosts,transactions
0,645048906856594097,https://www.airbnb.com/rooms/645048906856594097,20220620202126,2022-06-20,Furnished 1 bedroom condo with beautiful view,🏢Condo For rent: Lumpini Ville Nakhon In-River...,Its in Lumpini building,https://a0.muscache.com/pictures/miso/Hosting-...,4663219,"Tambon Talat Kwan, Chang Wat Nonthaburi, Thailand",...,,,,,,,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 4663219,""host_name"" : ""Fawad"",""ho...",
1,15636062,https://www.airbnb.com/rooms/15636062,20220620202126,2022-06-20,Akesin place ngamwongwan,ที่ของฉันใกล้กับร้านอาหารและของกิน และ ขนส่งสา...,,https://a0.muscache.com/pictures/f68098b9-3ff2...,100840975,,...,,,,,,,f,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 100840975,""host_name"" : ""Supika"",...",
2,39544047,https://www.airbnb.com/rooms/39544047,20220620202126,2022-06-20,IMPACT Thunder Dome Muangthong ND condo ( C1 ),,IMPACT ศูนย์แสดงสินค้า 550 m<br />IMPACT Chal...,https://a0.muscache.com/pictures/cc410e7d-922f...,303573998,"Tambon Ban Mai, จ.นนทบุรี, Thailand",...,5.0,5.0,5.0,5.0,4.67,4.67,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 303573998,""host_name"" : ""Noodee"",...","[{""date"" : ""2019-11-15"",""reviewer_id"" : ""27394..."
3,34002624,https://www.airbnb.com/rooms/34002624,20220620202126,2022-06-20,Popular Condo Near IMPACT,คอนโดเมืองทองอยู่ในบริเวณเดียวกับศูนย์แสดงสินค...,คอนโดเมืองทองอยู่ในบริเวณเดียวกับศูนย์แสดงสินค...,https://a0.muscache.com/pictures/e4811699-0153...,256747704,"ตำบล บางพูด, Chang Wat Nonthaburi, Thailand",...,5.0,5.0,5.0,4.0,4.5,4.0,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 256747704,""host_name"" : ""Krit"",""h...","[{""date"" : ""2019-06-01"",""reviewer_id"" : ""25693..."
4,22678899,https://www.airbnb.com/rooms/22678899,20220620202126,2022-06-20,บ้านเบนซ์,เป็นหมู่บ้านขนาดใหญ่มีหลายเฟส บ้านเป็นทาวน์โฮม...,,https://a0.muscache.com/pictures/02e4ae53-dbe6...,167136634,,...,,,,,,,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 167136634,""host_name"" : ""เบนซ์"",""...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17069,22947580,https://www.airbnb.com/rooms/22947580,20220620202126,2022-06-20,Apartment T9 Muang Thong Thani by KhunNutt2B7,ห้องพักส่วนตัวสวยหรูอยู่สบาย สะอาด ราคาประหยัด...,ที่พักอยู่ใกล้ห้างคอสโม แหล่งช้อปปิ้ง Sport Fa...,https://a0.muscache.com/pictures/2cbf3a10-76e8...,169985358,"Tambon Ban Mai, Chang Wat Nonthaburi, Thailand",...,,,,,,,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 169985358,""host_name"" : ""Nath"",""h...",
17070,5955949,https://www.airbnb.com/rooms/5955949,20220620202126,2022-06-20,Cozy Manor house with Lake view 520 square meters,Nice place for relax with wide lake at opposit...,There are many Thai restaurant near entrance o...,https://a0.muscache.com/pictures/93a7608f-f35c...,20002101,"Tambon Bang Nam Chut, Chang Wat Samut Sakhon, ...",...,,,,,,,f,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 20002101,""host_name"" : ""Nutchuda""...",
17071,32650300,https://www.airbnb.com/rooms/32650300,20220620202126,2022-06-20,Studio Superior (Double Bed) Room 1,- Room size 25 sq.m. or 322 sq.ft.<br />- Quee...,"Bangna Tower<br />40,000 square meters project...",https://a0.muscache.com/pictures/9a88370c-111c...,245251233,"บางแก้ว, Samut Prakan, Thailand",...,,,,,,,t,"[{""date"" : ""2022-06-20"",""available"" : ""t"",""pri...","{""host_id"" : 245251233,""host_name"" : ""Bangna P...",
17072,46851152,https://www.airbnb.com/rooms/46851152,20220620202126,2022-06-20,Family Room Dmk Do

In [40]:
def csv_to_json(csvf, jsonFilePath):
    jsonArray = []
    for index, row in csvf.iterrows():
        datadict = {}
        for col in csvf.columns.tolist():
            if col not in ['dates','hosts','transactions']:
                datadict[col] = row[col] 
            else:
                try:
                    datadict[col] = json.loads(row[col])
                except:
                    pass
        #print(json.dumps(datadict, indent=2, sort_keys=True))
        jsonArray.append(datadict)

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
            jsonString = json.dumps(jsonArray, indent=4)
            jsonf.write(jsonString)

In [44]:
csv_to_json(listings_df2, 'transformed.json')

#### check

In [45]:
f = open('transformed.json')
Lines = f.readlines() 

In [47]:
for i in range(0,10000):
    print(Lines[i])

[

    {

        "listing_id": 645048906856594097,

        "listing_url": "https://www.airbnb.com/rooms/645048906856594097",

        "scrape_id": 20220620202126,

        "last_scraped": "2022-06-20T00:00:00.000Z",

        "name": "Furnished 1 bedroom condo with beautiful view",

        "description": "\ud83c\udfe2Condo For rent: Lumpini Ville Nakhon In-Riverview Condo \ud83c\udfe2<br /><br /> \ud83d\ude98 Only 1.3 km. from the Purple Line  \ud83d\ude84 Yaek Tiwanon Station, near Lotus and Makro, Nakhon In and BigC Tiwanon<br /><br /><b>The space</b><br />32 sqm<br /><br /><b>Guest access</b><br />You will get the entire condo access alont with elevator card",

        "neighborhood_overview": "Its in Lumpini building",

        "picture_url": "https://a0.muscache.com/pictures/miso/Hosting-645048906856594097/original/c235b616-353f-486c-84ce-2fec5048587d.jpeg",

        "host_id": 4663219,

        "neighbourhood": "Tambon Talat Kwan, Chang Wat Nonthaburi, Thailand",

        "neig

                "available": "t",

                "price": 2500.0,

                "adjusted_price": 2500.0,

                "minimum_nights": 2.0,

                "maximum_nights": 1125.0

            },

            {

                "date": "2022-09-26T00:00:00.000Z",

                "available": "t",

                "price": 2500.0,

                "adjusted_price": 2500.0,

                "minimum_nights": 2.0,

                "maximum_nights": 1125.0

            },

            {

                "date": "2022-09-27T00:00:00.000Z",

                "available": "t",

                "price": 2500.0,

                "adjusted_price": 2500.0,

                "minimum_nights": 2.0,

                "maximum_nights": 1125.0

            },

            {

                "date": "2022-09-28T00:00:00.000Z",

                "available": "t",

                "price": 2500.0,

                "adjusted_price": 2500.0,

                "minimum_nights": 2.0,

              


            {

                "date": "2022-11-19T00:00:00.000Z",

                "available": "f",

                "price": 600.0,

                "adjusted_price": 600.0,

                "minimum_nights": 15.0,

                "maximum_nights": 1125.0

            },

            {

                "date": "2022-11-20T00:00:00.000Z",

                "available": "f",

                "price": 600.0,

                "adjusted_price": 600.0,

                "minimum_nights": 15.0,

                "maximum_nights": 1125.0

            },

            {

                "date": "2022-11-21T00:00:00.000Z",

                "available": "f",

                "price": 600.0,

                "adjusted_price": 600.0,

                "minimum_nights": 15.0,

                "maximum_nights": 1125.0

            },

            {

                "date": "2022-11-22T00:00:00.000Z",

                "available": "f",

                "price": 600.0,

                "adjusted_price"